# DER
Dark Experience - catastrophic experience forgetting

In [1]:
!git clone https://github.com/esthy13/cil-intrusion-detection

fatal: destination path 'cil-intrusion-detection' already exists and is not an empty directory.


In [3]:
%cd cil-intrusion-detection
!git pull

/content/cil-intrusion-detection
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 14 (delta 3), reused 14 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (14/14), 17.37 MiB | 12.29 MiB/s, done.
From https://github.com/esthy13/cil-intrusion-detection
   060afc2..320a8dc  main       -> origin/main
Updating 060afc2..320a8dc
Fast-forward
 .env                                   |      2 -
 README.md                              |     27 +
 data/CICIDS2017.csv                    | 225746 ++++++++++++++++++++++++++++++
 modules/preprocessing.py               |      0
 modules/strategies/der.py              |      0
 modules/strategies/er.py               |      0
 modules/strategies/icarl.py            |      0
 modules/training.py                    |      0
 Esther.ipynb => notebooks/Esther.ipynb |     71 +-
 modules/metrics.py => requirements.txt |      0
 src/data.py                 

In [6]:
!ls src

data.py  train.py


In [5]:
import sys, os
sys.path.append("/content/cil-intrusion-detection/src")

from src.data import trial

ModuleNotFoundError: No module named 'src.data'

## 1. Downloading dataset
Download the dataset from https://www.kaggle.com/datasets/chethuhn/network-intrusion-dataset

## 2. Explore the dataset

In [ ]:
import pandas as pd
df_2017 = pd.read_csv("dataset/CICIDS2017.csv")
df_2017.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


The column label declares whether certain values in the dataset are related to an attack or not

In [ ]:
df_2017.describe()

c:\Users\giuli\Documents\ai-uni-projects\cil-intrusion-detection\venv\Lib\site-packages\pandas\core\nanops.py:1020: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\giuli\Documents\ai-uni-projects\cil-intrusion-detection\venv\Lib\site-packages\pandas\core\nanops.py:1020: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,225745.00000,2.257450e+05,225745.000000,225745.000000,225745.000000,2.257450e+05,225745.000000,225745.000000,225745.000000,225745.000000,...,225745.000000,225745.000000,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05
mean,8879.61946,1.624165e+07,4.874916,4.572775,939.463346,5.960477e+03,538.535693,27.882221,164.826715,214.907242,...,3.311497,21.482753,1.848261e+05,1.293436e+04,2.080849e+05,1.776201e+05,1.032214e+07,3.611943e+06,1.287813e+07,7.755355e+06
std,19754.64740,3.152437e+07,15.422874,21.755356,3249.403484,3.921834e+04,1864.128991,163.324159,504.892965,797.411073,...,12.270018,4.166799,7.979250e+05,2.102737e+05,9.002350e+05,7.842602e+05,2.185303e+07,1.275689e+07,2.692126e+07,1.983109e+07
min,0.00000,-1.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,80.00000,7.118000e+04,2.000000,1.000000,26.000000,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.00000,1.452333e+06,3.000000,4.000000,30.000000,1.640000e+02,20.000000,0.000000,8.666667,5.301991,...,2.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,80.00000,8.805237e+06,5.000000,5.000000,63.000000,1.160100e+04,34.000000,6.000000,32.000000,10.263203,...,4.000000,20.000000,1.878000e+03,0.000000e+00,1.878000e+03,1.862000e+03,8.239725e+06,0.000000e+00,8.253838e+06,7.422849e+06
max,65532.00000,1.199999e+08,1932.000000,2942.000000,183012.000000,5.172346e+06,11680.000000,1472.000000,3867.000000,6692.644993,...,1931.000000,52.000000,1.000000e+08,3.950000e+07,1.000000e+08,1.000000e+08,1.200000e+08,6.530000e+07,1.200000e+08,1.200000e+08


There are no null values in the rows of the dataset

In [ ]:
df_2017.index

Cloning into 'cil-intrusion-detection'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 38 (delta 14), reused 29 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 17.50 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (14/14), done.
